# User Interface 

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly_usmap import usmap_default
from plotly_usmap import usmap_choose
from plotly_usmap import newdf

## 1. Set up options for Dropdowns and Slider

In [2]:
rank = pd.read_csv('../data/rank_file.csv')

# Create a list of indicators 
available = list(rank.columns.values)
for i in ['Unnamed: 0','City','State','Population','Natural_total_rank', 'Human_related_rank', 
          'Economy_rank', 'Tertiary_Rank', 'Latitude', 'Longitude']:
    available.remove(i)
    
# Create a list of labels for dropdown
labels = ['Fewer Crimes','More Hospitals','Early Education Options',
          'University Options','Air Quality','Water Quality',
           'Fewer Toxics','Fewer Hazardous Particles','Green Coverage',
           'Employment Rate','Sales Revenue','Income','Tuition Affordability',
          'Bars','Restaurants','Museums','Libraries','Parks','Top Restaurants']

# Put available and labels in a two-dimensional list
pairs = [available,labels]

## 2. Set up layout for User Interface

In [3]:
app = dash.Dash()
# Get DataFrame value
import UI_setup
app.layout = UI_setup.layout_setup(pairs)

## 3.Set up for default graph

In [4]:
@app.callback(
    dash.dependencies.Output('Total-graphic', 'figure'),
    [dash.dependencies.Input('Total', 'n_clicks')]
)
def total_graph(Total):
    if(Total is None):
        return
    return {
           usmap_default(rank)    
        }
@app.callback(
    dash.dependencies.Output('Human-graphic', 'figure'),
    [dash.dependencies.Input('Hrank', 'n_clicks')]
)
def human_graph(Hrank):
    if(Hrank is None):
        return
    return {
           usmap_default(rank, 'human')    
        }
@app.callback(
    dash.dependencies.Output('Natural-graphic', 'figure'),
    [dash.dependencies.Input('Nrank', 'n_clicks')]
)
def environment_graph(Nrank):
    if(Nrank is None):
        return
    return {
           usmap_default(rank, 'natural')    
        }
@app.callback(
    dash.dependencies.Output('Economic-graphic', 'figure'),
    [dash.dependencies.Input('Erank', 'n_clicks')]
)
def economy_graph(Erank):
    if(Erank is None):
        return
    return {
           usmap_default(rank, 'economy')    
        }
@app.callback(
    dash.dependencies.Output('Tertiary-graphic', 'figure'),
    [dash.dependencies.Input('Trank', 'n_clicks')]
)
def entertainment_graph(Trank):
    if(Trank is None):
        return
    return {
           usmap_default(rank, 'tertiary')    
        }

## 4.Set up for user call

In [5]:
@app.callback(
    dash.dependencies.Output('User-graphic', 'figure'),
    [dash.dependencies.Input('Search', 'n_clicks')],
    [dash.dependencies.State('First-care', 'value'),
     dash.dependencies.State('Second-care', 'value'),
     dash.dependencies.State('Third-care', 'value'),
     dash.dependencies.State('Fourth-care', 'value'),
     dash.dependencies.State('Fifth-care', 'value'),
    ]
)
    
def user_DIY_graph(Search, First_care, Second_care, Third_care, Fourth_care, Fifth_care):
    df = newdf(rank, First_care, Second_care, Third_care, Fourth_care, Fifth_care)
    
    return {
        usmap_choose(df)
    }


## 5. Running

In [ ]:
app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Dec/2017 02:35:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2017 02:35:16] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2017 02:35:16] "GET /_dash-dependencies HTTP/1.1" 200 -
[2017-12-13 02:35:17,273] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.d